In [1]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aceeae4e-1a97-4690-b577-a2c79d85f00b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 220ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      d

In [3]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/stats/"

client.delete(to_path,True)

True

In [4]:
# read data from the silver tables
df = spark.table("silver.stats")

In [5]:
df.printSchema()
df.count()

root
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: double (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = true)
 |-- OBPM: double (nullable = true)
 |-- DBPM: double (nullable = true)
 |-- BPM: double (nullable = true)


25475

In [6]:
# apagar todos os dados anteriores à introdução dos triplos
gd_df = df.where("Year > 1979")

In [7]:
gd_df = gd_df.withColumn("Year", gd_df["Year"].cast(IntegerType()))

In [8]:
gd_df.printSchema()
gd_df.count()

root
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: double (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = true)
 |-- OBPM: double (nullable = true)
 |-- DBPM: double (nullable = true)
 |-- BPM: double (nullable = true)

19778

In [9]:
# apagar coluna da idade pois já existe no gd_physical
# gd_df = gd_df.select("Year","Player","Pos","Tm","Team","G","GS","MP","PER","TSP","3PAr","FTr","ORBP","DRBP","TRBP","ASTP","STLP","BLKP","TOVP","USGP","OWS","DWS","WS","WS48","OBPM","DBPM","BPM","VORP","FG","FGA","FGP","3P","3PA","3PP","2P","2PA","2PP","eFGP","FT","FTA","FTP","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS")

In [10]:
gd_df.createOrReplaceTempView("gd_df")

spark.sql(
"""
SELECT 3P from gd_df WHERE Year=1980
"""
).show()

22/01/15 15:03:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+
|  3P|
+----+
| 0.0|
| 0.0|
| 0.0|
| 4.0|
| 0.0|
| 1.0|
| 0.0|
|16.0|
| 1.0|
| 0.0|
|73.0|
| 0.0|
| 0.0|
| 0.0|
| 8.0|
| 0.0|
| 1.0|
| 0.0|
| 1.0|
|10.0|
+----+
only showing top 20 rows



In [11]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/stats/")

In [12]:
# check the results in the table
spark.table("gold.stats").show()
spark.table("gold.stats").count()

+----+-----------------+---+---+---+----+---+---+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+----+---+---+------+----+----+----+----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+------+
|Year|           Player|Pos|Age| Tm|Team|  G| GS|  MP| PER|  TSP| 3PAr|  FTr|ORBP|DRBP|TRBP|ASTP|STLP|BLKP|TOVP|USGP| OWS|DWS| WS|  WS48|OBPM|DBPM| BPM|VORP|   FG|   FGA|  FGP|   3P|  3PA|  3PP|   2P|   2PA|  2PP| eFGP|   FT|  FTA|  FTP|  ORB|  DRB|  TRB|  AST|  STL| BLK|  TOV|   PF|   PTS|
+----+-----------------+---+---+---+----+---+---+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+----+---+---+------+----+----+----+----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+------+
|2015|       Quincy Acy| PF| 24|NYK|null| 68| 22|1287|11.9|0.533|0.181|0.293| 6.9|20.5|13.5| 8.7| 1.1| 1.4|13.8|15.5| 1.0|0.

19778